In [37]:
import pandas as pd
import os
# load: 
import os 
import json
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

In [3]:
# read_csvs: 
train_path = '/home/ec2-user/SageMaker/Kartik_Amazon_Thesis/general_training_data'

os.listdir(train_path)

['zero_star_general.csv',
 'disagreement_with_ratings_general.csv',
 'wrong_buying_general.csv',
 'rating_managment_explicit_general.csv']

In [4]:
rating_management_df = pd.read_csv(os.path.join(train_path,'rating_managment_explicit_general.csv'))
disgreement_df = pd.read_csv(os.path.join(train_path,'disagreement_with_ratings_general.csv'))
wrong_buying_df = pd.read_csv(os.path.join(train_path,'wrong_buying_general.csv'))
zero_star_df= pd.read_csv(os.path.join(train_path,'zero_star_general.csv'))

In [27]:
zero_star_df

,review_id,full_review,zero_star,zero_star_ohe
0,R1CRQ4TO7TVZYI,So the image shows noodlers polar ink series a...,0.0,0
1,R19JEPGLD3YJSH,I don’t think I have yet to write a negative r...,0.0,0
2,RITV3I7304IAG,Doesn’t erase. Even after I used some elbow gr...,0.0,0
3,R2C71Z7CM7WQ8O,Sadly I wish I had read the reviews before buy...,0.0,0
4,RGF63NTC51ASI,The media could not be loaded. ...,4.0,1
...,...,...,...,...
11225,R3P1FAGQ8FVCW6,I have never written a negative review but thi...,0.0,0
11226,R30Y7QFKEJXKU3,"Other reviewers say the same thing, so I don’t...",0.0,0
11227,RVXZJB60W7RE1,I bought based on reviews. My Epson R280 has ...,0.0,0
11228,R20ZK2HFCH5FL4,"the reviewers talk about tape, markers and bub...",0.0,0


In [28]:
merged_df = rating_management_df.merge(disgreement_df[['review_id', 'disagreement_with_ratings_ohe']], on = 'review_id')
merged_df= merged_df.merge(wrong_buying_df[['review_id', 'wrong_buying_ohe']], on = 'review_id')
merged_df= merged_df.merge(zero_star_df[['review_id', 'zero_star_ohe']], on = 'review_id')
merged_df

,review_id,full_review,rating_managment_explicit,rating_managment_explicit_ohe,disagreement_with_ratings_ohe,wrong_buying_ohe,zero_star_ohe
0,R1CRQ4TO7TVZYI,So the image shows noodlers polar ink series a...,0,0,0,0,0
1,R19JEPGLD3YJSH,I don’t think I have yet to write a negative r...,0,0,0,0,0
2,RITV3I7304IAG,Doesn’t erase. Even after I used some elbow gr...,1,1,0,0,0
3,R2C71Z7CM7WQ8O,Sadly I wish I had read the reviews before buy...,0,0,0,0,0
4,RGF63NTC51ASI,The media could not be loaded. ...,0,0,0,0,1
...,...,...,...,...,...,...,...
11225,R3P1FAGQ8FVCW6,I have never written a negative review but thi...,0,0,0,0,0
11226,R30Y7QFKEJXKU3,"Other reviewers say the same thing, so I don’t...",0,0,0,0,0
11227,RVXZJB60W7RE1,I bought based on reviews. My Epson R280 has ...,0,0,0,1,0
11228,R20ZK2HFCH5FL4,"the reviewers talk about tape, markers and bub...",0,0,0,0,0


In [29]:
def combine_all_tags(rating_management, disagreement, zero_stars, wrong_buying): 
    if rating_management ==1 or disagreement ==1 or zero_stars ==1 or wrong_buying ==1: 
        return 1 
    return 0 

In [39]:
merged_df['combine_tag']= merged_df.apply(lambda x: combine_all_tags(x.rating_managment_explicit_ohe, x.disagreement_with_ratings_ohe, x.wrong_buying_ohe, x.zero_star_ohe), axis =1)

In [40]:
merged_df.combine_tag.value_counts()

0    8229
1    3001
Name: combine_tag, dtype: int64

In [38]:
# load the pre-trained models: 
from transformers import RobertaTokenizer, TFRobertaModel
with tf.device('/device:GPU:0'):
    tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
    model = TFRobertaModel.from_pretrained("roberta-base")

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [41]:
# train test split - split trainning into train and validation, take 10% as validation
train, test_df = train_test_split(merged_df, test_size= 0.3, random_state=42, stratify=merged_df['combine_tag'])

In [42]:
# train test split - split trainning into train and validation, take 10% as validation
train_df_1, val_df = train_test_split(train, test_size= 0.3, random_state=42, stratify=train['combine_tag'])

In [43]:

class create_tensor_dataset(): 
    
    '''Purpose: Create a tensorflow dataset for model training
    df_to_convert: The dataframe of interest, that contains the 
    text columns
    text: The column within the dataframe containing the text 
    batch_size: int, batch size
    tokenizer: The tokenizer to be utilised
    labels: The class/label column within the dataframe
    '''
    
    def __init__(self, df_to_convert, text,target_col, batch_size, labels=None ): 
        self.df_to_convert = df_to_convert
        self.text = text # column containing the text (here chunked_data)
        self.target_col = target_col # y column of interest here label
        self.batch_size = batch_size
        self.tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
        self.labels = labels # list of all the labels 
        
    def generate_training_data(self, ids, masks):
        for i, text in tqdm(enumerate(self.df_to_convert[self.text])):
            with tf.device('/device:GPU:0'):
                
                tokenized_text =  self.tokenizer.encode_plus(
                    text,# The sequence or batch of sequences to be encoded.
                    max_length=256, #Controls the maximum length to use by one of the truncation/padding parameters.
                    truncation=True, # Truncate to a maximum length specified with the argument max_length
                    padding='max_length', #pad to a maximum length specified with the argument max_length 
                    add_special_tokens=True,# encode the sequences with the special tokens relative to their model.
                    return_tensors='tf'   #return TensorFlow tf.constant objects.
                )
                ids[i, :] = tokenized_text.input_ids ##List of token ids to be fed to a model
                masks[i, :] = tokenized_text.attention_mask #List of indices specifying which tokens should be attended to by the model
        return ids, masks
    
    def ohe_label(self): 
         # convert label values to a list 
        return self.df_to_convert[self.target_col].values.tolist()

    def DatasetMapFunction(self, input_ids, attn_masks, labels):
        # create and return key-value pair of input ids, attention mask and the corresnponding labels
        return {
            'input_ids': input_ids,
            'attention_mask': attn_masks
        }, labels
    
    def tensor_data(self): 
        # input_ids array
        X_input_ids = np.zeros((len(self.df_to_convert), 256))
        # attention mask array
        X_attn_masks = np.zeros((len(self.df_to_convert), 256))
        # retrieve the input ids and attention masks
        X_input_ids, X_attn_masks = self.generate_training_data(X_input_ids, X_attn_masks)
        # creation of tensorflow dataset: 
        dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, self.ohe_label()))
        dataset = dataset.map(self.DatasetMapFunction)
        dataset = dataset.shuffle(10000).batch(self.batch_size, drop_remainder=True)
        return dataset

In [44]:
# creation of tensorflow dataset for training
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
train_tensor =create_tensor_dataset(train_df_1, 'full_review', 'combine_tag',8 )
train_dataset =train_tensor.tensor_data() 

5502it [00:06, 849.74it/s]


In [45]:
# validation tensor dataset: 
with tf.device('/device:GPU:0'):
    val_tensor =create_tensor_dataset(val_df, 'full_review', 'combine_tag', 8)
    val_dataset =val_tensor.tensor_data() 

2359it [00:02, 795.89it/s]


In [46]:

# defining 2 input layers for input_ids and attn_masks
with tf.device('/device:GPU:0'):
    input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
    attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')
    bert_embds = model.roberta(input_ids, attention_mask=attn_masks)[1] # pooled output
    # intermediate_layer_1 = tf.keras.layers.Dropout(0.5, name="dropout")(bert_embds)# 0 -> activation layer (3D), 1 -> pooled output layer (2D)
    intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
#     intermediate_layer_2 = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer_2')(intermediate_layer)
    # intermediate_layer_2 = tf.keras.layers.Dropout(0.4, name="dropout2")(intermediate_layer)
    output_layer = tf.keras.layers.Dense(1, activation='sigmoid', name='output_layer')(intermediate_layer) # sigmoid -> calcs probs of binary classes
    roberta_imdb = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
    roberta_imdb.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 roberta (TFRobertaMainLayer)   TFBaseModelOutputWi  124645632   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [47]:
# utilization of adam optimization 
optim = tf.keras.optimizers.Adam(learning_rate=1e-5)
# utilised binary cross entropy loss 
loss_func = tf.keras.losses.BinaryCrossentropy()
roberta_imdb.compile(optimizer=optim, loss=loss_func, metrics=['accuracy'])

# early stopping - administer val_loss: 
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping()
custom_early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience= 3,
    mode='min'
)

with tf.device('/device:GPU:0'):

    hist = roberta_imdb.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=100, 
        callbacks =[custom_early_stopping]
    )

Epoch 1/100
687/687 [==============================] - 362s 481ms/step - loss: 0.3889 - accuracy: 0.8373 - val_loss: 0.2920 - val_accuracy: 0.8886
Epoch 2/100
687/687 [==============================] - 324s 472ms/step - loss: 0.1946 - accuracy: 0.9361 - val_loss: 0.2461 - val_accuracy: 0.9048
Epoch 3/100
687/687 [==============================] - 324s 472ms/step - loss: 0.1363 - accuracy: 0.9567 - val_loss: 0.3011 - val_accuracy: 0.9090
Epoch 4/100
687/687 [==============================] - 323s 471ms/step - loss: 0.0908 - accuracy: 0.9740 - val_loss: 0.3661 - val_accuracy: 0.8967
Epoch 5/100
687/687 [==============================] - 323s 471ms/step - loss: 0.0657 - accuracy: 0.9796 - val_loss: 0.3849 - val_accuracy: 0.9128


In [48]:
def prepare_data(input_text, tokenizer):
    with tf.device('/device:GPU:0'):
        
        token = tokenizer.encode_plus(
            input_text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        return {
            'input_ids': tf.cast(token.input_ids, tf.float64),
            'attention_mask': tf.cast(token.attention_mask, tf.float64)
        }

def make_predictions(model, input_text,threshold, label_list=None ): 
    with tf.device('/device:GPU:0'):
        processed_data = prepare_data(input_text, tokenizer)
        probs = model.predict(processed_data)
#         print(probs)
        if probs[0]> threshold: 
            return label_list[0]
        else: 
            return label_list[1]
#     return(probs[0])

In [49]:
# tt = test_df.head(20)
label_list= [1, 0]
test_df['result']= test_df.full_review.apply(lambda x: make_predictions(roberta_imdb, x, 0.5, label_list))

1/1 [==============================] - 0s 62ms/step


1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 59ms/step


1/1 [==============================] - 0s 61ms/step


...

1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 61ms/step


1/1 [==============================] - 0s 67ms/step


In [50]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
print (classification_report(test_df['combine_tag'], test_df['result']))

              precision    recall  f1-score   support

           0       0.94      0.93      0.94      2469
           1       0.81      0.85      0.83       900

    accuracy                           0.91      3369
   macro avg       0.88      0.89      0.88      3369
weighted avg       0.91      0.91      0.91      3369



In [51]:
roberta_imdb.save('all_tags_final')

INFO:tensorflow:Assets written to: all_tags_final/assets


INFO:tensorflow:Assets written to: all_tags_final/assets
